# WTI vs Brent Crude Spread Analysis

**Analyzing the price differential between West Texas Intermediate (WTI) and Brent Crude oil benchmarks**

## What You'll Learn
- Fetch historical oil price data using [OilPriceAPI](https://oilpriceapi.com)
- Calculate and visualize the WTI-Brent spread
- Understand what drives price differentials
- Identify trading opportunities

## About the Data
This notebook uses the [OilPriceAPI Python SDK](https://github.com/oilpriceapi/python-sdk) to fetch historical data.

**Get your free API key**: [oilpriceapi.com/auth/signup](https://oilpriceapi.com/auth/signup) (1,000 requests/month free)

**GitHub**: [OilpriceAPI/kaggle-notebooks](https://github.com/OilpriceAPI/kaggle-notebooks)

## Setup & Installation

In [ ]:
!pip install oilpriceapi pandas matplotlib seaborn -q

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from oilpriceapi import OilPriceAPI

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

print("✅ Libraries loaded")

## Initialize API Client

**On Kaggle:** Add your API key as a secret with label `OILPRICEAPI_KEY`  
**Locally:** Set `OILPRICEAPI_KEY` environment variable or pass directly

In [ ]:
# Get API key from Kaggle secrets or environment
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("OILPRICEAPI_KEY")
    print("✅ Using Kaggle secrets")
except:
    api_key = os.environ.get('OILPRICEAPI_KEY')
    print("✅ Using environment variable")

# Initialize client with longer timeout for Kaggle environment
client = OilPriceAPI(api_key=api_key, timeout=120)
print("✅ API client initialized")

## Fetch Historical Data

Using the SDK's built-in `to_dataframe()` method for easy data loading.

In [ ]:
# Define date range (past year)
end_date = datetime.now()
start_date = end_date - timedelta(days=365)

print(f"Fetching data from {start_date.date()} to {end_date.date()}")
print("\nFetching WTI data...")
wti_df = client.historical.to_dataframe(
    commodity='WTI_USD',
    start=start_date,
    end=end_date,
    interval='daily'
)

print("Fetching Brent data...")
brent_df = client.historical.to_dataframe(
    commodity='BRENT_CRUDE_USD',
    start=start_date,
    end=end_date,
    interval='daily'
)

print(f"\n✅ Fetched {len(wti_df)} WTI prices")
print(f"✅ Fetched {len(brent_df)} Brent prices")

## Data Preparation

In [ ]:
# Rename columns for clarity
wti_df = wti_df[['value']].rename(columns={'value': 'WTI'})
brent_df = brent_df[['value']].rename(columns={'value': 'Brent'})

# Merge on index (dates)
df = wti_df.join(brent_df, how='inner')

# Calculate spread (Brent - WTI)
df['Spread'] = df['Brent'] - df['WTI']
df['Spread_Pct'] = (df['Spread'] / df['WTI']) * 100

print("\n📊 Data Summary:")
print(df.describe())

## Visualization 1: Price Trends

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Plot 1: Absolute Prices
ax1.plot(df.index, df['WTI'], label='WTI', linewidth=2, color='#1f77b4')
ax1.plot(df.index, df['Brent'], label='Brent Crude', linewidth=2, color='#ff7f0e')
ax1.set_ylabel('Price (USD/barrel)', fontsize=12)
ax1.set_title('WTI vs Brent Crude Oil Prices (Past Year)', fontsize=14, fontweight='bold')
ax1.legend(loc='best', fontsize=11)
ax1.grid(True, alpha=0.3)

# Plot 2: Spread
ax2.plot(df.index, df['Spread'], linewidth=2, color='#2ca02c')
ax2.axhline(y=0, color='red', linestyle='--', alpha=0.7, label='Zero spread')
ax2.fill_between(df.index, df['Spread'], 0, where=(df['Spread'] > 0), 
                 alpha=0.3, color='green', label='Brent premium')
ax2.fill_between(df.index, df['Spread'], 0, where=(df['Spread'] < 0), 
                 alpha=0.3, color='red', label='WTI premium')
ax2.set_ylabel('Spread (USD/barrel)', fontsize=12)
ax2.set_xlabel('Date', fontsize=12)
ax2.set_title('Brent-WTI Spread', fontsize=14, fontweight='bold')
ax2.legend(loc='best', fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Current Spread: ${df['Spread'].iloc[-1]:.2f}/barrel")
print(f"📊 Average Spread: ${df['Spread'].mean():.2f}/barrel")
print(f"📊 Spread Range: ${df['Spread'].min():.2f} to ${df['Spread'].max():.2f}")

## Visualization 2: Spread Distribution

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Histogram
ax1.hist(df['Spread'], bins=30, color='steelblue', edgecolor='black', alpha=0.7)
ax1.axvline(df['Spread'].mean(), color='red', linestyle='--', linewidth=2, 
           label=f'Mean: ${df["Spread"].mean():.2f}')
ax1.axvline(df['Spread'].median(), color='green', linestyle='--', linewidth=2, 
           label=f'Median: ${df["Spread"].median():.2f}')
ax1.set_xlabel('Spread (USD/barrel)', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.set_title('Distribution of Brent-WTI Spread', fontsize=13, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Box plot
ax2.boxplot(df['Spread'], vert=True, patch_artist=True,
           boxprops=dict(facecolor='lightblue', color='blue'),
           medianprops=dict(color='red', linewidth=2),
           whiskerprops=dict(color='blue'),
           capprops=dict(color='blue'))
ax2.set_ylabel('Spread (USD/barrel)', fontsize=12)
ax2.set_title('Spread Statistics', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Conclusion & Resources

This analysis demonstrates the historical relationship between WTI and Brent crude prices and how to identify trading opportunities.

### Learn More
- **API Documentation**: [docs.oilpriceapi.com](https://docs.oilpriceapi.com)
- **Python SDK**: [github.com/oilpriceapi/python-sdk](https://github.com/oilpriceapi/python-sdk)
- **Get Free API Key**: [oilpriceapi.com/auth/signup](https://oilpriceapi.com/auth/signup)
- **This Notebook on GitHub**: [github.com/OilpriceAPI/kaggle-notebooks](https://github.com/OilpriceAPI/kaggle-notebooks)

---

*Data provided by [OilPriceAPI](https://oilpriceapi.com) - Professional commodity price data at 98% less cost than Bloomberg Terminal*